In [1]:
import librosa
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

import IPython.display as ipd

In [2]:
path = 'C:/Users/muham.LAPTOP-MJ49C84G/Downloads/ub'

df = pd.read_csv('Downloads/ub/UrbanSound8K.csv')

In [3]:
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [6]:
df['class'].unique()

array(['dog_bark', 'children_playing', 'car_horn', 'air_conditioner',
       'street_music', 'gun_shot', 'siren', 'engine_idling', 'jackhammer',
       'drilling'], dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8732 entries, 0 to 8731
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   slice_file_name  8732 non-null   object 
 1   fsID             8732 non-null   int64  
 2   start            8732 non-null   float64
 3   end              8732 non-null   float64
 4   salience         8732 non-null   int64  
 5   fold             8732 non-null   int64  
 6   classID          8732 non-null   int64  
 7   class            8732 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 545.9+ KB


In [8]:
# different classes data in different folds
appended = []
for i in range(1,11):
    appended.append(df[df.fold == i]['class'].value_counts())

class_distribution = pd.DataFrame(appended)
class_distribution = class_distribution.reset_index()
class_distribution['index'] = ["fold"+str(x) for x in range(1,11)]
class_distribution

,index,jackhammer,dog_bark,children_playing,street_music,air_conditioner,drilling,engine_idling,siren,car_horn,gun_shot
0,fold1,120,100,100,100,100,100,96,86,36,35
1,fold2,120,100,100,100,100,100,100,91,42,35
2,fold3,120,100,100,100,100,100,107,119,43,36
3,fold4,120,100,100,100,100,100,107,166,59,38
4,fold5,120,100,100,100,100,100,107,71,98,40
5,fold6,68,100,100,100,100,100,107,74,28,46
6,fold7,76,100,100,100,100,100,106,77,28,51
7,fold8,78,100,100,100,100,100,88,80,30,30
8,fold9,82,100,100,100,100,100,89,82,32,31
9,fold10,96,100,100,100,100,100,93,83,33,32


In [4]:
def features_extract(file):
    audio,sample_rate = librosa.load(file_name,res_type='kaiser_fast')
    feature = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=50)
    scaled_feature = np.mean(feature.T,axis=0)
    return scaled_feature

extracted = []

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['slice_file_name'])) 
    final_class_labels = row['class']   
    data= features_extract(file_name)    
    extracted.append([data,final_class_labels])

3555it [10:37,  5.02it/s]C:\Users\muham.LAPTOP-MJ49C84G\anaconda3\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8326it [26:11,  1.83it/s]C:\Users\muham.LAPTOP-MJ49C84G\anaconda3\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
8329it [26:11,  3.16it/s]C:\Users\muham.LAPTOP-MJ49C84G\anaconda3\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [27:21,  5.32it/s]


In [5]:
ext_df = pd.DataFrame(extracted,columns=['feature','class'])
ext_df

,feature,class
0,"[-217.35526, 70.22338, -130.38527, -53.282898,...",dog_bark
1,"[-424.09818, 109.34077, -52.919525, 60.86475, ...",children_playing
2,"[-458.79114, 121.38419, -46.520657, 52.00812, ...",children_playing
3,"[-413.89984, 101.66373, -35.42945, 53.036354, ...",children_playing
4,"[-446.60352, 113.68541, -52.402206, 60.302044,...",children_playing
...,...,...
8727,"[-398.5845, 135.53496, -50.725018, 35.855907, ...",car_horn
8728,"[-346.4742, 86.34815, -45.16858, 51.709873, -3...",car_horn
8729,"[-303.88824, 111.35945, -45.941563, 35.877018,...",car_horn
8730,"[-344.11008, 125.45021, -54.903442, 34.891148,...",car_horn


In [9]:
x = np.array(ext_df['feature'].tolist())
y = np.array(ext_df['class'].tolist())

In [10]:
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)
print("Number of training samples = ", x_train.shape[0])
print("Number of testing samples = ",x_test.shape[0])

Number of training samples =  6985
Number of testing samples =  1747


In [12]:
num_labels = y.shape[1]
model = Sequential()

model.add(Dense(128, input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               6

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

In [14]:
num_epochs = 150
num_batch_size = 32

model.fit(
          x_train, 
          y_train, 
          batch_size=num_batch_size, 
          epochs=num_epochs,
          validation_data=(x_test, y_test),
         )

Epoch 1/150
219/219 [==============================] - 5s 10ms/step - loss: 8.5190 - accuracy: 0.1487 - val_loss: 2.0846 - val_accuracy: 0.2919
Epoch 2/150
219/219 [==============================] - 2s 7ms/step - loss: 2.2914 - accuracy: 0.2215 - val_loss: 2.0201 - val_accuracy: 0.2261
Epoch 3/150
219/219 [==============================] - 2s 7ms/step - loss: 2.0445 - accuracy: 0.2686 - val_loss: 1.9380 - val_accuracy: 0.2776
Epoch 4/150
219/219 [==============================] - 2s 8ms/step - loss: 1.9286 - accuracy: 0.3051 - val_loss: 1.7645 - val_accuracy: 0.4064
Epoch 5/150
219/219 [==============================] - 2s 8ms/step - loss: 1.8430 - accuracy: 0.3372 - val_loss: 1.6833 - val_accuracy: 0.4316
Epoch 6/150
219/219 [==============================] - 2s 8ms/step - loss: 1.7348 - accuracy: 0.3808 - val_loss: 1.5310 - val_accuracy: 0.5117
Epoch 7/150
219/219 [==============================] - 2s 8ms/step - loss: 1.6638 - accuracy: 0.4067 - val_loss: 1.4353 - val_accuracy: 0.519

In [15]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T,axis=0)
    return np.array([scaled])

In [16]:
def print_prediction(file_name):
    pred_fea = extract_feature(file_name) 
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    print("The predicted class is:", pred_class[0], '\n') 

In [18]:
file_name = 'Downloads/ub/fold1/101415-3-0-2.wav'

print_prediction(file_name)

ipd.Audio(file_name)

1/1 [==============================] - 0s 324ms/step
The predicted class is: dog_bark 



In [22]:
file_name = 'Downloads/ub/fold7/102853-8-0-4.wav'

print_prediction(file_name)

ipd.Audio(file_name)

1/1 [==============================] - 0s 57ms/step
The predicted class is: siren 



In [23]:
file_name = 'Downloads/ub/fold9/105029-7-3-0.wav'

print_prediction(file_name)

ipd.Audio(file_name)

1/1 [==============================] - 0s 65ms/step
The predicted class is: jackhammer 



In [24]:
file_name = 'Downloads/car-horn-beep-beep-two-beeps-honk-honk-6188.mp3'

print_prediction(file_name)

ipd.Audio(file_name)

1/1 [==============================] - 0s 195ms/step
The predicted class is: car_horn 

